In [10]:
#!pip install tensorflow
import pandas as pd

In [42]:
corpus1 = pd.read_csv('박원순.csv', sep=",")
corpus1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24984 entries, 0 to 24983
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Video ID  24984 non-null  object
 1   Title     24984 non-null  object
 2   Comment   24984 non-null  object
dtypes: object(3)
memory usage: 585.7+ KB


In [45]:
corpus1.Comment

0                   아이구~이건머~~~쓰레기  종합선물셋트 구만.....참~나라 꼬라지~
1        인생은 그리길지않다\r\n살아온날보다 살아갈날이 얼마남지\r\n안았는데 추미애 너무...
2                                                   개판 나라다
3        안죽었다면. 장례식은 뭔가요\r\n대사관쪽애서 사라졌다고 하는대. 미리 찻속애서 일...
4                                 조용히 가족장으로 지내야지 뭔 시장장 웃기네
                               ...                        
24979                          편히  쉬세요.  우리 시장님!  지켜봐  주세요
24980                                       삼가 고인의 명복을 빕니다
24981     하늘에선 부디  평안하시길!  우리  깨시민들이  꼭  당신의  결백을 밝혀내겠습니다!
24982                                       삼가 고인의 명복을 빕니다
24983                            '그 고소인의 면모'가 서서히 드러나고 있다.
Name: Comment, Length: 24984, dtype: object

In [12]:
import tensorflow as tf
#tf.__version__

'2.2.0'

In [17]:
def remove_stop_words(corpus):
    stop_words = ['개', '쌍']
    results = []
    for text in corpus:
        tmp = text.split(' ')
        for stop_word in stop_words:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))
    
    return results

In [30]:
corpus = remove_stop_words(corpus)


['Video ID', 'Title', 'Comment']

In [19]:
words = []
for text in corpus:
    for word in text.split(' '):
        words.append(word)

words = set(words)
words

{'Comment', 'ID', 'Title', 'Video'}

In [22]:
word2int = {}

for i,word in enumerate(words):
    word2int[word] = i

sentences = []

for sentence in corpus:
    sentences.append(sentence.split())
    
WINDOW_SIZE = 2

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1] : 
            if neighbor != word:
                data.append([word, neighbor])

In [23]:
import pandas as pd
for text in corpus:
    print(text)

df = pd.DataFrame(data, columns = ['input', 'label'])

df.head(10)

Video ID
Title
Comment


,input,label
0,Video,ID
1,ID,Video


In [24]:
word2int

{'Comment': 0, 'Title': 1, 'Video': 2, 'ID': 3}

In [25]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

ONE_HOT_DIM = len(words)

# function to convert numbers to one hot vectors
def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

X = [] # input word
Y = [] # target word

for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word2int[ x ]))
    Y.append(to_one_hot_encoding(word2int[ y ]))

# convert them to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)

# making placeholders for X_train and Y_train
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

# word embedding will be 2 dimension for 2d visualization
EMBEDDING_DIM = 2 

# hidden layer: which represents word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1])) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function: cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

# training operation
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
init = tf.global_variables_initializer()
sess.run(init) 

iteration = 30000
for i in range(iteration):
    # input is X_train which is one hot encoded word
    # label is Y_train which is one hot encoded neighbor word
    sess.run(train_op, feed_dict={x: X_train, y_label: Y_train})
    if i % 3000 == 0:
        print('iteration '+str(i)+' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))

In [ ]:
vectors = sess.run(W1 + b1)
print(vectors)

In [ ]:
w2v_df = pd.DataFrame(vectors, columns = ['x1', 'x2'])
w2v_df['word'] = words
w2v_df = w2v_df[['word', 'x1', 'x2']]
w2v_df